In [ ]:
#Getting the ID of the EntitySchema
id = input("What is the ID of the considered EntitySchema")
url = "https://www.wikidata.org/wiki/Special:EntitySchemaText/"+ id

What is the ID of the considered EntitySchemaE105


In [ ]:
#Retrieving the EntitySchema
import requests
entityschema = requests.get(url)
schematext = entityschema.text

In [ ]:
schematext

'PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX pr: <http://www.wikidata.org/prop/reference/>\nPREFIX pq: <http://www.wikidata.org/prop/qualifier/>\nPREFIX ps: <http://www.wikidata.org/prop/statement/>\nPREFIX wdref: <http://www.wikidata.org/reference/>\nPREFIX p: <http://www.wikidata.org/prop/>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\nPREFIX wd: <http://www.wikidata.org/entity/>\nPREFIX wikibase: <http://wikiba.se/ontology#>\nPREFIX prov: <http://www.w3.org/ns/prov#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\n# example SPARQL: SELECT ?wdItem_overledene_op_nlWikipedia WHERE { ?artikel_over_overledene_op_nlWikipedia schema:isPartOf <https://nl.wikipedia.org/> ; schema:about ?wdItem_overledene_op_nlWikipedia . ?wdItem_overledene_op_nlWikipedia  wdt:P31 wd:Q5 ; wdt:P570 ?dod .} LIMIT 100\n\nstart=@<deceased_person> \n\n<deceased_person> EXTRA p:P31 {\n\twdt:P31   [wd:Q5] ;\n\tp:P570  @<P570_dod> ;\n\tp

In [ ]:
#Extracting the EntitySchema input
def extractvar(schematext):
  begin = max(schematext.find("start ="), schematext.find("start="))
  varlist = schematext[begin:]
  varlist = varlist[:varlist.find("\n")]
  return list(map(lambda x: x.split(), varlist.split("@")))[1:]

In [ ]:
var = extractvar(schematext)
var = [x[0] for x in var]
print(var)

['<deceased_person>']


In [ ]:
#Extracting the EntitySchema statements
def extractstatements(schematext):
  begin = max(schematext.find("start ="), schematext.find("start="))
  varlist = schematext[begin:]
  varlist = varlist[varlist.find("\n"):]
  return varlist

In [ ]:
#Getting the subjects of the ShEx statements
c = extractstatements(schematext)
subjectvar = []
subjectstatements = []
while c != "":
  subjectvar.append(c[:c.find("{")])
  c = c[c.find("{")+1:]
  n = 1
  p = 0
  while (n > 0) and (p < len(c)):
    if c[p] == "{": n+=1
    if c[p] == "}": n-=1
    p += 1
  subjectstatements.append(c[:p])
  c = c[p:]

In [ ]:
subjectvar = list(map(lambda x: x.strip(), subjectvar))
subjectvar = list(map(lambda x: x[:x.find(">")]+">", subjectvar))
subjectvar = [x.replace("#","") for x in subjectvar]
print(subjectvar)

['<deceased_person>', '<P570_dod>', '<P569_dob>', '<P27_citizenship>', '<main_reference>', '<country>']


In [ ]:
#Getting the predicates and objects of the ShEx statements
stlist = list(map(lambda x: x.split("\n"), subjectstatements))
print(stlist)
cleanstatements = []
for i in stlist:
  j = [x.strip() for x in i]
  cleanstatements.append([x for x in j if (x.find("wd:Q")>=0 or x.find("@")>=0)])

[['', '\twdt:P31   [wd:Q5] ;', '\tp:P570  @<P570_dod> ;', '\tp:P569  @<P569_dob>? ;', '\tp:P27   @<P27_citizenship>+ ;', '}'], ['', '\tps:P570 xsd:dateTime ;', '\t#prov:wasDerivedFrom @<main_reference>+ ; ', '}'], ['', '\tps:P569 xsd:dateTime ;', '\t#prov:wasDerivedFrom @<main_reference>+ ; ', '}'], ['', '\tps:P27  @<country> ;', '\t#prov:wasDerivedFrom @<main_reference>+ ;', '}'], ['', '\tpr:P248 {} ;', '\tpr:P577 {} ;', '\tpr:P854 {} ;', '\tpr:P1476 {} ;', '\tpr:P6333 {} ;', '\tpr:P813 {} ;', '}'], ['', '\twdt:P31 [wd:Q6256] ;', '}']]


In [ ]:
while (cleanstatements.count([])>0):
  uvar = subjectvar[cleanstatements.index([])]
  clean = []
  for i in cleanstatements:
    clean.append([x for x in i if (x.find(uvar) == -1)])
  cleanstatements = clean
  del subjectvar[cleanstatements.index([])]
  del cleanstatements[cleanstatements.index([])]

In [ ]:
clean = []
for l in cleanstatements:
  list01 = []
  for s in l:
    if s.find("@") == -1:
      list01.append(s)
    else:
      cc = 0
      for element in subjectvar:
        if (s.find(element) >= 0): cc += 1
      if cc > 0: list01.append(s)
  clean.append(list01)
cleanstatements = clean

In [ ]:
clean = []
for l in cleanstatements:
  clean.append([x.replace("{", "$").replace("}", "£") for x in l])
cleanstatements = clean

In [ ]:
#Formulating the SPARQL query to extract the Wikidata ID of the entities to be validated
for element in var:
  subjectvar = [x.replace(element, "<id>") for x in subjectvar]
subjectvar = [x.replace("<", "?") for x in subjectvar]
subjectvar = [x.replace(">", "") for x in subjectvar]

clean = []
for l in cleanstatements:
  list01 = [x.replace("?","") for x in l]
  list01 = [x.replace("@<","?") for x in list01]
  list01 = [x.replace(">","") for x in list01]
  list01 = [x.replace("+","") for x in list01]
  list01 = [x.replace("*","") for x in list01]
  list01 = [x[:-1] for x in list01]
  clean.append(list01)
cleanstatements = clean

In [ ]:
statements = []
for i in range(len(cleanstatements)):
  list01 = [subjectvar[i] + " " + x for x in cleanstatements[i]]
  statements.append(list01)

In [ ]:
clean = []
for l in statements:
  list02 = []
  for element in l:
    if (element.find("[") >= 0):
      base = element[:element.find("[")]
      variables = element[element.find("["):]
      vars = variables[variables.find("[")+1:variables.find("]")].strip().split(" ")
      string = "{"
      for var in vars:
        if (len(string) > 1): string += " UNION {"
        string += base+var+"}"
    else:
      string = element
    list02.append(string)
  clean.append(list02)
statements = clean
print(statements)

[['{?id wdt:P31   wd:Q5}', '?id p:P27   ?P27_citizenship '], ['?P27_citizenship ps:P27  ?country '], ['{?country wdt:P31 wd:Q6256}']]


In [ ]:
clean = []
for i in statements:
  clean.append([x for x in i if (":P" in x)])
statements = clean

clean = []
for i in statements:
  list01 = []
  for elements in i:
    if ("$" in elements) and not("£" in element):
      if (elements[-1] == "}"):
        list01.append(elements[:-1]+"£" * (elements[:-1].count("$") - elements[:-1].count("£")) +"}")
      else:
        list01.append(elements+"£" * (elements[:-1].count("$") - elements[:-1].count("£")))
    else:
      list01.append(elements)
  clean.append(list01)
statements = clean

clean = []
for i in statements:
  list01 = [x for x in i if not("EXTRA" in x)]
  clean.append(list01)
statements = clean
statements = [x for x in statements if x != []]
print(statements)

[['{?id wdt:P31   wd:Q5}', '?id p:P27   ?P27_citizenship '], ['?P27_citizenship ps:P27  ?country '], ['{?country wdt:P31 wd:Q6256}']]


In [ ]:
query = "SELECT ?id WHERE {\n"
if (subjectvar.count("?id") > 1):
  idstatements = []
  otherstatements = []
  for n in range(len(subjectvar)):
    if subjectvar[n] != "?id":
      otherstatements += statements[n]
    else:
      idstatements.append(statements[n].join(".\n"))
  query += "{" + idstatements.join("} UNION {\n") + "}"
  query += otherstatements.join(".\n")
else:
  for x in statements:
    query += "\n".join(x)+".\n"
query += "}"
query = query.replace("$", "[").replace("£", "]").replace("?#", "?")

In [ ]:
#Installing WikibaseIntegrator
!pip install SPARQLWrapper

In [ ]:
#Getting the IDs of the entities to be validated
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
offset = 0
limit = 100
queryf = query + "\nOFFSET " + str(offset) + "\nLIMIT " + str(limit)

def get_results(endpoint_url, queryf):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(queryf)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, queryf)

for result in results["results"]["bindings"]:
    print(result)


{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1554673'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3312010'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6794399'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6949475'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q54134095'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q65454793'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q65454796'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q65454799'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q633'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1276'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18804'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18947'}}
{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18950'}}
{'id': {'type': 'uri'

In [ ]:
query

'SELECT ?id WHERE {\n{?id wdt:P31   wd:Q5}\n?id p:P27   ?P27_citizenship .\n?P27_citizenship ps:P27  ?country .\n{?country wdt:P31 wd:Q6256}.\n}'